In [1]:
import os
import urllib.request
import pandas as pd

'''
WARNING: This is a lot of data and used
to just download the data. All the big cat related
csv files are already included in bigCatsCsv.
'''

#THIS IS TRAINING DATA AND CLASSIFICATION ITEMS

#Download CSV data to get some data.
def dl_if_not_exists(url,filename):
    if not os.path.isfile(filename):
        urllib.request.urlretrieve(url, filename)
    
dl_if_not_exists("https://storage.googleapis.com/openimages/2018_04/train/train-images-boxable-with-rotation.csv", "train-images-boxable-with-rotation.csv")
dl_if_not_exists("https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv",'class-descriptions-boxable.csv')
dl_if_not_exists("https://storage.googleapis.com/openimages/v5/train-annotations-human-imagelabels-boxable.csv",'train-annotations-human-imagelabels-boxable.csv')
dl_if_not_exists("https://storage.googleapis.com/openimages/v6/oidv6-train-annotations-bbox.csv",'oidv6-train-annotations-bbox.csv')

imagesId = pd.read_csv('train-images-boxable-with-rotation.csv',index_col=0)
imageLabels = pd.read_csv('train-annotations-human-imagelabels-boxable.csv',index_col=0)
boundingBoxes = pd.read_csv('oidv6-train-annotations-bbox.csv',index_col=0)
classDescriptions = pd.read_csv('class-descriptions-boxable.csv',header=None) \
                        .rename({0:'LabelName',1:'Class'},axis=1)

In [2]:
import os
import urllib.request
import pandas as pd

#THIS IS VALIDATION DATA

#Download CSV data to get some data.
def dl_if_not_exists(url,filename):
    if not os.path.isfile(filename):
        urllib.request.urlretrieve(url, filename)
        
#WARNING: This is a lot of data. Skip to the third cell if you just want to download the pictures.
dl_if_not_exists("https://storage.googleapis.com/openimages/v5/validation-annotations-bbox.csv", 
                 "validation-annotations-bbox.csv")
dl_if_not_exists("https://storage.googleapis.com/openimages/v5/validation-annotations-human-imagelabels-boxable.csv",
                 'validation-annotations-human-imagelabels-boxable.csv')
dl_if_not_exists("https://storage.googleapis.com/openimages/2018_04/validation/validation-images-with-rotation.csv",
                'validation-images-with-rotation.csv')

imagesId = pd.read_csv('validation-images-with-rotation.csv',index_col=0)
imageLabels = pd.read_csv('validation-annotations-human-imagelabels-boxable.csv',index_col=0)
boundingBoxes = pd.read_csv("validation-annotations-bbox.csv",index_col=0)
classDescriptions = pd.read_csv('class-descriptions-boxable.csv',header=None) \
                        .rename({0:'LabelName',1:'Class'},axis=1)

In [6]:
import os
import urllib.request
import pandas as pd

#THIS IS TESTING DATA

#Download CSV data to get some data.
def dl_if_not_exists(url,filename):
    if not os.path.isfile(filename):
        urllib.request.urlretrieve(url, filename)
        
#WARNING: This is a lot of data. Skip to the third cell if you just want to download the pictures.
dl_if_not_exists("https://storage.googleapis.com/openimages/v5/test-annotations-bbox.csv", 
                 "test-annotations-bbox.csv")
dl_if_not_exists("https://storage.googleapis.com/openimages/v5/test-annotations-human-imagelabels-boxable.csv",
                 'test-annotations-human-imagelabels-boxable.csv')
dl_if_not_exists("https://storage.googleapis.com/openimages/2018_04/test/test-images-with-rotation.csv",
                'test-images-with-rotation.csv')

imagesId = pd.read_csv('test-images-with-rotation.csv',index_col=0)
imageLabels = pd.read_csv('test-annotations-human-imagelabels-boxable.csv',index_col=0)
boundingBoxes = pd.read_csv("test-annotations-bbox.csv",index_col=0)
classDescriptions = pd.read_csv('class-descriptions-boxable.csv',header=None) \
                        .rename({0:'LabelName',1:'Class'},axis=1)

In [7]:
from functools import reduce
Urls = imagesId[['OriginalURL',"OriginalLandingURL"]]

#These are big cats image ids.
catsSet = ['/m/07dm6','/m/096mb','/m/0c29q','/m/0cd4d']

dfs = []
for c in catsSet:
    dfs.append(imageLabels[imageLabels['LabelName'] == c])
    
filteredLabels = reduce(lambda a,b : a.append(b),dfs)
filteredLabels = filteredLabels[filteredLabels['Confidence'] == 1][['LabelName']]

boundingBoxes = boundingBoxes[['LabelName','XMin','XMax','YMin','YMax']]

dfs = []
for c in catsSet:
    dfs.append(boundingBoxes[boundingBoxes['LabelName'] == c])
    
filteredBoxes = reduce(lambda a,b : a.append(b),dfs)

In [8]:
import pandas as pd
import urllib.request
import os
import asyncio
import shutil
df1 = filteredLabels.join(Urls)
df2 = df1.merge(filteredBoxes,on=['ImageID','LabelName'])
df3 = df2.merge(classDescriptions,on=['LabelName'])
df3.pop('LabelName')
df3.pop('OriginalLandingURL')
df3.to_csv('big_cats_image_dataTesting.csv')
files = pd.read_csv('big_cats_image_dataTesting.csv',header=[0])

#Download CSV data to get some data.
async def dl_if_not_exists_async(url):
    try:
        filename = "./bigCats/"+url[url.rfind('/')+1:]
        if not os.path.isfile(filename):
            urllib.request.urlretrieve(url, filename)
    except:
        pass
            

'''
for r in files.iloc:
    print(r.name,r.OriginalURL)
'''

if not os.path.isdir('bigCats'):
    os.mkdir('bigCats', 0o755)

tasks = [dl_if_not_exists_async(r.OriginalURL) for r in files.iloc]
await asyncio.wait(tasks)

({<Task finished coro=<async-def-wrapper.<locals>.dl_if_not_exists_async() done, defined at <ipython-input-8-d1eb1815998c>:18> result=None>,
  <Task finished coro=<async-def-wrapper.<locals>.dl_if_not_exists_async() done, defined at <ipython-input-8-d1eb1815998c>:18> result=None>,
  <Task finished coro=<async-def-wrapper.<locals>.dl_if_not_exists_async() done, defined at <ipython-input-8-d1eb1815998c>:18> result=None>,
  <Task finished coro=<async-def-wrapper.<locals>.dl_if_not_exists_async() done, defined at <ipython-input-8-d1eb1815998c>:18> result=None>,
  <Task finished coro=<async-def-wrapper.<locals>.dl_if_not_exists_async() done, defined at <ipython-input-8-d1eb1815998c>:18> result=None>,
  <Task finished coro=<async-def-wrapper.<locals>.dl_if_not_exists_async() done, defined at <ipython-input-8-d1eb1815998c>:18> result=None>,
  <Task finished coro=<async-def-wrapper.<locals>.dl_if_not_exists_async() done, defined at <ipython-input-8-d1eb1815998c>:18> result=None>,
  <Task finis

In [15]:
files = [pd.read_csv('big_cats_image_data.csv',header=[0],index_col=0),
        pd.read_csv('big_cats_image_dataValidation.csv',header=[0],index_col=0),
        pd.read_csv('big_cats_image_dataTesting.csv',header=[0],index_col=0)]
allCats = reduce(lambda a,b : a.append(b),files)
allCats = allCats.reset_index(drop=True)

allCats.to_csv('AllBigCats.csv')

,OriginalURL,XMin,XMax,YMin,YMax,Class
0,https://c1.staticflickr.com/8/7315/12483130855...,0.370625,0.962500,0.152622,0.905431,Leopard
1,https://farm2.staticflickr.com/3710/1164444715...,0.232500,0.924375,0.300188,0.939963,Leopard
2,https://farm4.staticflickr.com/7589/1664394915...,0.000000,0.999375,0.141667,0.944167,Leopard
3,https://c3.staticflickr.com/1/523/18089504933_...,0.000000,0.925000,0.070812,0.835924,Leopard
4,https://farm3.staticflickr.com/5018/5546319570...,0.000000,0.869141,0.071615,0.998698,Leopard
...,...,...,...,...,...,...
5060,https://c8.staticflickr.com/3/2018/2097656877_...,0.000000,0.865625,0.000000,1.000000,Tiger
5061,https://c3.staticflickr.com/6/5497/12266437605...,0.162242,0.672566,0.079646,0.851770,Tiger
5062,https://farm5.staticflickr.com/3535/3235685295...,0.000000,1.000000,0.000000,1.000000,Tiger
5063,https://c6.staticflickr.com/4/3179/2825352510_...,0.000000,1.000000,0.054167,1.000000,Tiger
